In [1]:
import torch
import torch.nn as nn # torch.nn module, contains classes and functions to help build neural networks

import torch.optim as optim # provides various optimization algorithms, such as SGD (Stochastic Gradient Descent), Adam, etc
from torch.utils.data import DataLoader, TensorDataset # Dataloader - helps to load data, TensorDataset - allows to access data points as pairs(i/p, target)

In [2]:
class MyNetwork(nn.Module):

    def __init__(self, input_dim=20, hidden_dim1=64, hidden_dim2=128, hidden_dim3=64, output_dim=1): # number of neurons in each layers, o/p-binary classification
        super(MyNetwork, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim1) #self is the keyword
        self.layer2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.output_layer = nn.Linear(hidden_dim1 + hidden_dim3, output_dim)
        self.relu = nn.ReLU()  # used after each hidden layer
        self.sigmoid = nn.Sigmoid() # o/p considering as 0 or 1

    def forward(self, x): # x is the i/p, can have multiple inputs - x1,x2..
        # First layer
        out1 = self.relu(self.layer1(x))

        # Second layer
        out2 = self.relu(self.layer2(out1))

        # Third layer
        out3 = self.relu(self.layer3(out2))

        # Concatenate outputs of the 1st and 3rd layers
        concatenated_output = torch.cat((out1, out3), dim=1)

        # Output layer
        output = self.sigmoid(self.output_layer(concatenated_output))

        return output

In [3]:
x = torch.randn(32, 20)  # Batch of 32 samples with 20 features (dimension) each
model = MyNetwork()
output = model(x)
print(output.shape)  # Should print torch.Size([32, 1])

torch.Size([32, 1])


In [4]:
print(model)

MyNetwork(
  (layer1): Linear(in_features=20, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [11]:
def generate_synthetic_data(num_samples=1000, input_dim=20):
    # Generate random features
    X = torch.randn(num_samples, input_dim) # generates a tensor of shape (num_samples, input_dim) filled with random numbers drawn from a normal distribution

    # Generate random labels (0 or 1)
    y = torch.randint(0, 2, (num_samples, 1)).float() # generates a tensor of shape (num_samples, 1) filled with random integers between 0 and 1

    return X, y


In [12]:
# training loop for a neural network model in PyTorch
# criterion - include nn.CrossEntropyLoss() for classification or nn.MSELoss() for regression.
# num_epochs - number of times to iterate over the entire training dataset

def train_model(model, dataloader, criterion, optimizer, num_epochs=10, device='cpu'):
    model.train() # sets the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device) # inputs and labels are moved to the specified device cpu/gpu

            # Zero the parameter gradients
            optimizer.zero_grad() # Before performing the forward pass, the gradients from the previous iteration are cleared

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels) # loss is calculated by comparing the model's outputs to the true labels

            # Backward pass and optimize
            loss.backward() # computes the gradients of the loss
            optimizer.step() # updates the model's parameters

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

    print('Training complete.')


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
# Hyperparameters
input_dim = 20
hidden_dim1 = 64
hidden_dim2 = 128
hidden_dim3 = 64
output_dim = 1
batch_size = 32
learning_rate = 0.001 # controlling the step size during gradient descent
num_epochs = 10

# Initialize the model, criterion, and optimizer
model = MyNetwork(input_dim=input_dim, hidden_dim1=hidden_dim1, hidden_dim2=hidden_dim2, hidden_dim3=hidden_dim3, output_dim=output_dim).to(device)
criterion = nn.BCELoss() # Binary Cross-Entropy Loss - used for binary classification

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Generate synthetic data
X, y = generate_synthetic_data(num_samples=1000, input_dim=input_dim)

# Create DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # data is shuffled at the beginning of each epoch - generalize better

# Train the model
train_model(model, dataloader, criterion, optimizer, num_epochs=num_epochs, device=device)

Epoch [1/10], Loss: 0.6969
Epoch [2/10], Loss: 0.6883
Epoch [3/10], Loss: 0.6781
Epoch [4/10], Loss: 0.6686
Epoch [5/10], Loss: 0.6544
Epoch [6/10], Loss: 0.6292
Epoch [7/10], Loss: 0.5957
Epoch [8/10], Loss: 0.5684
Epoch [9/10], Loss: 0.5174
Epoch [10/10], Loss: 0.4909
Training complete.


In [15]:
def evaluate_model(model, dataloader, device='cpu'):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No need to calculate gradients during evaluation
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()  # Apply threshold for binary classification
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # Accuracy Calculation

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')

def make_inference(model, input_data, device='cpu'):
    model.eval()  # Set the model to evaluation mode
    input_data = input_data.to(device)
    with torch.no_grad():  # No need to calculate gradients during inference
        output = model(input_data)
        predicted = (output > 0.5).float()  # Apply threshold for binary classification
    return predicted

# Example usage:

# Generate synthetic test data (similar to training data)
X_test, y_test = generate_synthetic_data(num_samples=200, input_dim=input_dim)

# Create DataLoader for test data
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Evaluate the model on the test data
evaluate_model(model, test_dataloader, device=device)

# Example of making predictions on a new input
new_input = torch.randn(1, input_dim)  # Generate a random new input sample
predicted_label = make_inference(model, new_input, device=device)
print(f'Predicted label: {predicted_label.item()}')


Accuracy: 45.50%
Predicted label: 1.0
